In [1]:
from taxcalc import *
from functions3 import *
import pandas as pd
import numpy as np
import copy

## Base Work
Read in the supplementary files, create the base calculator for comparison, determine totals for additional revenue

In [2]:
# Other benefits programs
other_programs = pd.read_csv('benefitprograms.csv')
other_programs['Cost'] *= 1000000
other_rev = other_programs['Cost'].sum()
other_rev

426839000000L

In [3]:
# CPS and associated weights file
cps = pd.read_csv('../taxdata/cps_data/cps_ben_full.csv')
cps_wt = pd.read_csv('../Dropbox/cps_weights.csv')
# Allocate benefits from other programs to individuals
cps['dist_ben'] = cps['mcaid'] + cps['ssi'] + cps['snap'] + cps['vb']
cps['ratio'] = cps.dist_ben * cps.s006 / (cps.dist_ben * cps.s006).sum()
cps['other'] = cps.ratio * other_programs['Cost'].sum() / cps.s006  # Why did I divide by s006?

In [4]:
# Base Calculator
recs = Records(cps, weights=cps_wt, adjust_ratios=None, start_year=2014)
pol = Policy(start_year=2014, num_years=11)
calc = Calculator(policy=pol, records=recs)
calc.records.ssi = cps['ssi']
calc.records.snap = cps['snap']
calc.records.mcare = cps['mcare']
calc.records.mcaid = cps['mcaid']
calc.records.vb = cps['vb']
calc.records.ss = cps['ss']
calc.records.total_ben = (calc.records.ssi + calc.records.snap + calc.records.mcare +
                          calc.records.mcaid + calc.records.vb + calc.records.ss)
calc.calc_all()
ctam_rev = (calc.records.total_ben * calc.records.s006).sum()
ctam_rev

1754648972214.849

In [5]:
# Calculate loss in social security revenue
r_ss = Records(cps, weights=cps_wt, adjust_ratios=None, start_year=2014)
pol_ss = Policy(start_year=2014, num_years=11)
c_ss = Calculator(records=r_ss, policy=pol_ss, verbose=False)
c_ss.records.e02400 = np.zeros(len(c_ss.records.e02400))
c_ss.advance_to_year(2014)
c_ss.calc_all()

# Lost Revenue
ss_lostrev = ((c_ss.records.combined - calc.records.combined) * c_ss.records.s006).sum()
ss_lostrev

-23429022651.484211

In [6]:
cps_storage = copy.deepcopy(cps)

# UBI With Tax Reform

In [7]:
# Calculator with original tax refrom
recs_reform = Records(cps, weights=cps_wt, adjust_ratios=None, start_year=2014)
pol_reform = Policy(start_year=2014, num_years=11)
tax_reform = {
    2014: {
        '_ALD_StudentLoan_hc': [1.0],
        '_ALD_SelfEmploymentTax_hc': [1.0],
        '_ALD_SelfEmp_HealthIns_hc': [1.0],
        '_ALD_KEOGH_SEP_hc': [1.0],
        '_ALD_EarlyWithdraw_hc': [1.0],
        '_ALD_Alimony_hc': [1.0],
        '_ALD_Dependents_hc': [1.0],
        '_ALD_EducatorExpenses_hc': [1.0],
        '_ALD_HSADeduction_hc': [1.0],
        '_ALD_IRAContributions_hc': [1.0],
        '_ALD_DomesticProduction_hc': [1.0],
        '_ALD_Tuition_hc': [1.0],
        '_CR_RetirementSavings_hc': [1.0],
        '_CR_ForeignTax_hc': [1.0],
        '_CR_ResidentialEnergy_hc': [1.0],
        '_CR_GeneralBusiness_hc': [1.0],
        '_CR_MinimumTax_hc': [1.0],
        '_CR_AmOppRefundable_hc': [1.0],
        '_CR_AmOppNonRefundable_hc': [1.0],
        '_CR_SchR_hc': [1.0],
        '_CR_OtherCredits_hc': [1.0],
        '_CR_Education_hc': [1.0],
        '_II_em': [0.0],
        '_STD': [[0.0, 0.0, 0.0, 0.0, 0.0]],
        '_STD_Aged': [[0.0, 0.0, 0.0, 0.0, 0.0]],
        '_ID_Medical_hc': [1.0],
        '_ID_StateLocalTax_hc': [1.0],
        '_ID_RealEstate_hc': [1.0],
        '_ID_InterestPaid_hc': [1.0],
        '_ID_Casualty_hc': [1.0],
        '_ID_Miscellaneous_hc': [1.0],
        '_CDCC_c': [0.0],
        '_CTC_c': [0.0],
        '_EITC_c': [[0.0, 0.0, 0.0, 0.0]],
        '_LLC_Expense_c': [0.0],
        '_ETC_pe_Single': [0.0],
        '_ETC_pe_Married': [0.0]
    }
}
pol_reform.implement_reform(tax_reform)
calc_reform = Calculator(records=recs_reform, policy=pol_reform, verbose=False)
calc_reform.records.e02400 = np.zeros(len(calc_reform.records.e02400))
calc_reform.calc_all()

In [8]:
tax_rev = ((calc_reform.records.combined - calc.records.combined) * calc_reform.records.s006).sum()
tax_rev

522512332558.16803

In [9]:
revenue = ctam_rev + other_rev + ss_lostrev + tax_rev
revenue

2680571282121.5332

In [10]:
# Number above and below 18
u18 = (calc_reform.records.nu18 * calc_reform.records.s006).sum()
abv18 = ((calc_reform.records.n1821 + calc_reform.records.n21) * calc_reform.records.s006).sum()

In [11]:
# Find original UBI amounts
ubi18, ubiu18 = ubi_amt(revenue, u18, abv18)
ubi18, ubiu18

(9611.2357485325665, 4805.6178742662833)

In [12]:
# Find UBI after accounting for UBI tax revenue
diff = 9e99
ubi_tax_rev = 0
prev_ubi_tax_rev = 0
while abs(diff) >= 100:
    ubi18, ubiu18 = ubi_amt(revenue + ubi_tax_rev, u18, abv18)
    diff, ubi_tax_rev = ubi_finder(ubi18, ubiu18, 
                                   tax_reform=tax_reform, revenue=revenue,
                                   calc_reform=calc_reform)
    if diff > 0:
        ubi_tax_rev = prev_ubi_tax_rev * 0.5
    prev_ubi_tax_rev = ubi_tax_rev
ubi18, ubiu18

(11914.98032791868, 5957.49016395934)

In [13]:
# Calculator with UBI and tax reform
recs_ubi1 = Records(cps, weights=cps_wt, adjust_ratios=None, start_year=2014)
pol_ubi1 = Policy(start_year=2014, num_years=11)
pol_ubi1.implement_reform(tax_reform)
ubi_ref = {
    2014: {
        '_UBI1': [ubiu18],
        '_UBI2': [ubi18],
        '_UBI3': [ubi18]
    }
}
pol_ubi1.implement_reform(ubi_ref)
calc_ubi1 = Calculator(records=recs_ubi1, policy=pol_ubi1, verbose=False)
calc_ubi1.records.e02400 = np.zeros(len(calc_ubi1.records.e02400))
calc_ubi1.advance_to_year(2014)
calc_ubi1.calc_all()

In [14]:
# Get MTR's
mtrs = calc_ubi1.mtr()

In [15]:
pd.options.display.float_format = '{:,.2f}'.format

## For all Tax Units

In [16]:
table_data1 = prep_table_data(calc=calc_ubi1, calc_base=calc, mtrs=mtrs, bins='income')

In [17]:
avg_ben, avg_ben_mult = cps_avg_ben(cps_storage, other_programs, group='all', bins='income')
table(table_data1, avg_ben, avg_ben_mult)

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change -Welfare Adj,Avg Combined Change,Avg Combined Change -Welfare Adj
0,0,61.70,1.55,"11,338",14.2%,14.4%,28.6%,"3,278","16,964","-25,389","-21,205","-11,703","-7,518"
1,10,16.25,1.76,"11,124",14.2%,15.1%,29.3%,"6,143","18,394","-9,823","-7,644","2,427","4,606"
2,20,15.78,1.84,"11,132",14.2%,18.0%,32.2%,"6,719","19,194","-7,748","-5,966","4,726","6,509"
3,30,12.87,1.94,"11,120",14.2%,21.0%,35.2%,"7,205","20,237","-6,402","-5,014","6,630","8,018"
4,40,10.28,2.03,"11,108",14.2%,21.7%,35.9%,"7,851","21,231","-5,674","-4,592","7,706","8,788"
5,50,17.58,2.23,"11,046",14.2%,23.2%,37.4%,"9,366","23,235","-4,916","-4,053","8,953","9,816"
6,75,11.00,2.51,"10,898",14.2%,24.0%,38.1%,"12,131","25,892","-4,000","-3,397","9,761","10,364"
7,100,14.47,2.75,"10,757",11.4%,26.1%,37.5%,"15,020","28,067","-3,506","-3,047","9,542","10,001"
8,200,3.20,2.93,"10,609",7.0%,33.0%,40.1%,"20,257","29,557","-3,429","-2,986","5,871","6,314"
9,"1,000",0.07,2.84,"10,634",6.4%,38.5%,44.9%,"45,254","28,538","-4,820","-4,070","-21,536","-20,785"


# Tax Units w/ Someone above 65

In [18]:
table_data2 = prep_table_data(calc=calc_ubi1, calc_base=calc, mtrs=mtrs, group='65 or over', bins='income')

In [19]:
avg_ben, avg_ben_mult = cps_avg_ben(cps_storage, other_programs, group='65 or over', bins='income')
table(table_data2, avg_ben, avg_ben_mult)

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change -Welfare Adj,Avg Combined Change,Avg Combined Change -Welfare Adj
0,0,26.61,1.48,"11,856",14.2%,14.8%,29.0%,"2,898","17,469","-36,102","-31,756","-21,531","-17,184"
1,10,1.49,1.75,"11,777",14.1%,16.0%,30.1%,"5,377","20,372","-29,721","-26,527","-14,726","-11,532"
2,20,1.23,1.82,"11,737",14.2%,17.6%,31.7%,"5,937","21,026","-28,547","-25,721","-13,458","-10,633"
3,30,0.96,1.93,"11,696",14.2%,19.5%,33.7%,"6,299","22,103","-28,365","-25,526","-12,561","-9,722"
4,40,0.81,1.93,"11,714",14.2%,21.1%,35.3%,"6,475","22,185","-27,363","-25,132","-11,653","-9,422"
5,50,1.39,1.99,"11,725",14.2%,23.4%,37.6%,"7,714","23,005","-25,888","-23,789","-10,597","-8,498"
6,75,0.79,2.10,"11,691",14.2%,24.1%,38.2%,"9,865","24,093","-24,287","-22,723","-10,059","-8,496"
7,100,1.03,2.18,"11,662",11.1%,26.6%,37.8%,"11,857","24,907","-24,580","-22,922","-11,530","-9,871"
8,200,0.26,2.25,"11,631",7.2%,33.3%,40.5%,"16,032","25,573","-23,581","-22,254","-14,040","-12,714"
9,"1,001",0.01,2.05,"11,704",6.9%,38.4%,45.2%,"38,646","23,642","-26,906","-25,345","-41,910","-40,349"


# Tax Units w/out Someone Over 65

In [20]:
table_data3 = prep_table_data(calc=calc_ubi1, calc_base=calc, mtrs=mtrs, group='under 65', bins='income')

In [21]:
avg_ben, avg_ben_mult = cps_avg_ben(cps_storage, other_programs, group='under 65', bins='income')
table(table_data3, avg_ben, avg_ben_mult)

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change -Welfare Adj,Avg Combined Change,Avg Combined Change -Welfare Adj
0,0,35.09,1.59,"10,944",14.1%,14.2%,28.3%,"3,566","16,581","-17,262","-13,201","-4,247",-186
1,10,14.76,1.76,"11,057",14.2%,15.1%,29.3%,"6,220","18,193","-7,809","-5,732","4,164","6,241"
2,20,14.55,1.84,"11,081",14.2%,18.0%,32.2%,"6,785","19,039","-5,993","-4,298","6,261","7,956"
3,30,11.92,1.94,"11,074",14.2%,21.1%,35.3%,"7,278","20,087","-4,629","-3,358","8,180","9,451"
4,40,9.47,2.04,"11,056",14.2%,21.7%,35.9%,"7,969","21,149","-3,812","-2,828","9,368","10,352"
5,50,16.19,2.25,"10,988",14.2%,23.2%,37.3%,"9,508","23,255","-3,114","-2,357","10,633","11,390"
6,75,10.21,2.54,"10,836",14.2%,23.9%,38.1%,"12,306","26,032","-2,428","-1,900","11,297","11,826"
7,100,13.44,2.79,"10,688",11.4%,26.1%,37.5%,"15,261","28,309","-1,895","-1,527","11,153","11,520"
8,200,2.94,2.99,"10,518",7.0%,33.0%,40.0%,"20,631","29,910","-1,638","-1,273","7,641","8,006"
9,"1,000",0.07,2.92,"10,529",6.4%,38.5%,44.9%,"45,897","29,015","-2,671","-2,000","-19,553","-18,882"
